In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re


/home/joannelam/Projects/my_project/lib/python2.7/site-packages/pandas/_libs/__init__.py:3: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/home/joannelam/Projects/my_project/lib/python2.7/site-packages/pandas/_libs/__init__.py:3: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/home/joannelam/Projects/my_project/lib/python2.7/site-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/home/joannelam/Projects/my_project/lib/python2.7/site-packages/pandas/__init__.py:26: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  from pandas._libs import (hashtable as 

/home/joannelam/Projects/my_project/lib/python2.7/site-packages/sklearn/feature_extraction/hashing.py:9: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 80
  from . import _hashing
/home/joannelam/Projects/my_project/lib/python2.7/site-packages/sklearn/feature_extraction/hashing.py:9: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 144
  from . import _hashing
/home/joannelam/Projects/my_project/lib/python2.7/site-packages/sklearn/utils/extmath.py:24: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 80
  from ._logistic_sigmoid import _log_logistic_sigmoid
/home/joannelam/Projects/my_project/lib/python2.7/site-packages/sklearn/utils/extmath.py:24: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 144
  from ._logistic_sigmoid import _log_logistic_sigmoid
/home/joannelam/Projects/my_project/lib/pyth

In [18]:
import jieba

In [31]:
data = pd.read_csv('ai_challenger_sentiment_analysis_trainingset_20180816/sentiment_analysis_trainingset.csv')

In [32]:
data = data[['content','others_overall_experience']]

In [33]:
data.head()

,content,others_overall_experience
0,"""吼吼吼，萌死人的棒棒糖，中了大众点评的霸王餐，太可爱了。一直就好奇这个棒棒糖是怎么个东西，...",1
1,"""第三次参加大众点评网霸王餐的活动。这家店给人整体感觉一般。首先环境只能算中等，其次霸王餐提...",1
2,"""4人同行 点了10个小吃\n榴莲酥 榴莲味道不足 松软 奶味浓\n虾饺 好吃 两颗大虾仁\...",0
3,"""之前评价了莫名其妙被删 果断继续差评！ 换了菜单 价格更低 开始砸牌子 但套餐还是有150...",-1
4,"""出乎意料地惊艳，椰子鸡清热降火，美容养颜，大大满足了爱吃火锅怕上火星人。椰子冻是帅帅的老板...",1


In [34]:
data = data[data.others_overall_experience != 0]
# data['content'] = data['content'].apply(lambda x: x.lower())
# data['content'] = data['content'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['others_overall_experience'] == 1].size)
print(data[ data['others_overall_experience'] == -1].size)

140140
18768


In [35]:
data.head()

,content,others_overall_experience
0,"""吼吼吼，萌死人的棒棒糖，中了大众点评的霸王餐，太可爱了。一直就好奇这个棒棒糖是怎么个东西，...",1
1,"""第三次参加大众点评网霸王餐的活动。这家店给人整体感觉一般。首先环境只能算中等，其次霸王餐提...",1
3,"""之前评价了莫名其妙被删 果断继续差评！ 换了菜单 价格更低 开始砸牌子 但套餐还是有150...",-1
4,"""出乎意料地惊艳，椰子鸡清热降火，美容养颜，大大满足了爱吃火锅怕上火星人。椰子冻是帅帅的老板...",1
5,"""烤鸭使用的蘸酱很地道，觉得烤鸭好不好吃在很大程度上都取决于这个酱料是否正宗。个人感觉他们家...",1


In [36]:
data['content'] = data['content'].apply(lambda x: " ".join(jieba.cut(x, cut_all=False)))

In [39]:
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ').replace(u'～', ' ').replace(u'，', ' ').replace(u'！', ' ')

In [40]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['content'].values)
X = tokenizer.texts_to_sequences(data['content'].values)
X = pad_sequences(X)

In [41]:
X.shape

(81564, 1173)

In [48]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1173, 128)         256000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 1173, 128)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 591       
Total params: 511,391
Trainable params: 511,391
Non-trainable params: 0
_________________________________________________________________
None


In [49]:
Y = pd.get_dummies(data['others_overall_experience']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

((54647, 1173), (54647, 3))
((26917, 1173), (26917, 3))


In [50]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
 - 3158s - loss: 0.2934 - acc: 0.9128
Epoch 2/7
 - 3006s - loss: 0.2200 - acc: 0.9382
Epoch 3/7
 - 3009s - loss: 0.2018 - acc: 0.9441
Epoch 4/7
 - 3010s - loss: 0.1852 - acc: 0.9497
Epoch 5/7
 - 3011s - loss: 0.1745 - acc: 0.9533
Epoch 6/7
 - 3014s - loss: 0.1655 - acc: 0.9568
Epoch 7/7
 - 3015s - loss: 0.1571 - acc: 0.9592
